# 7 УРОК ГРАФИКИ, ДАШБОРДЫ И МОНИТОРИНГИ

## 1

In [ ]:
Ваш менее опытный коллега не уверен, что его таблица retail действительно создалась и добавилась в базу данных. 

Напишите запрос, который создаст таблицу, если она еще не существует (IF NOT EXISTS), со следующими параметрами:

Название: retail
База: test
Столбцы:
    
- InvoiceNo – идентификатор заказа, строковое значение

- StockCode – идентификатор товара, строковое значение

- Description – описание товара, строковое значение

- Quantity – количество единиц товара, добавленных в корзину (особенность данных – значения могут быть отрицательными, 32 бита)

- InvoiceDate – дата и время создания заказа (транзакции), часовой пояс – 'Europe/London'

- UnitPrice – цена за единицу товара (особенность данных – значения могут быть отрицательными, точность – три знака после точки, 64 бита)

- CustomerID – идентификатор пользователя, целое положительное число, 32 бита.

- Country – страна, строковое значение. 

Движок: MergeTree
Сортировка: InvoiceDate, CustomerID
Hint: Decimal64

In [ ]:
CREATE TABLE IF NOT EXISTS test.retail_Kamsarin ( 
    InvoiceNo String, 
    StockCode String,
    Description String,
    Quantity Int32,
    InvoiceDate DateTime('Europe/London'),
    UnitPrice Decimal64(3),
    CustomerID UInt32,
    Country String
    ) 

ENGINE = MergeTree -- движок
ORDER BY (InvoiceDate, CustomerID)

##### ОТВЕТ в LMS

In [ ]:
CREATE TABLE IF NOT EXISTS test.retail ( 
    InvoiceNo String, 
    StockCode String,
    Description String,
    Quantity Int32,
    InvoiceDate DateTime('Europe/London'),
    UnitPrice Decimal64(3),
    CustomerID UInt32,
    Country String
    ) 

ENGINE = MergeTree 
ORDER BY (InvoiceDate, CustomerID)

## 2

In [ ]:
В следующих шагах используется default.retail – данные о транзакциях британского интернет-магазина 

в период с 1 декабря 2010 по 9 декабря 2011, где:

InvoiceNo – номер транзакции
StockCode – код товара
Description – описание товара
Quantity – количество единиц товара, добавленных в заказ
InvoiceDate – дата транзакции 
UnitPrice – цена за единицу товара
CustomerID – id клиента
Country – страна, где проживает клиент
Для выполнения заданий и построения графиков вам понадобится Redash. О том, как в него зайти, 

как строить графики и дашборды – в Notion.

 

## 3

In [ ]:
Для начала посмотрите, в каких странах имеется наибольшее число уникальных пользователей. Постройте барчарт, где:

по оси x – страна
по оси y – число пользователей
В качестве ответа укажите число уникальных пользователей из Великобритании. 

Note: запятые в Redash отделяют каждые три знака в числе (тысячные доли, миллионные и так далее). 
    
Например, 2,535,966 - это 2535966. В ответ надо писать число без запятых.

In [ ]:
SELECT Country,
    COUNT(DISTINCT(CustomerID)) AS Count_CustomerID
FROM default.retail
GROUP BY Country

##### ОТВЕТ: 3950

## 4

In [ ]:
Следующая задача – посмотреть на динамику изменения числа активных пользователей в месяц 
в Великобритании, Австралии и Нидерландах. Полученная вами результирующая таблица должна иметь 
вид: страна - число уникальных пользователей за определённый месяц.

MAU (monthly active users) – число уникальных пользователей за месяц. 
Активные пользователи – те, кто сделал хотя бы один заказ за выбранный промежуток времени (месяц).
В качестве ответа укажите наименьшее число пользователей за февраль 2011. 

Hint: toStartOfMonth()

In [ ]:
- **Как решить это задание?**
    
    В этом задании вам нужно:
    
    - Посчитать количество **уникальных** пользователей
    - При помощи `toStartOfMonth()` конвертировать дату платежа к **началу месяца**
    - В `WHERE` нужно отфильтровать страны United Kingdom, Netherlands и Australia
    - Визуализировать данные с помощью Line-chart, где по оси X - месяц, по оси Y - количество **уникальных** пользователей, 
    группировав по стране.
    
    Ваш лайн-чарт должен выглядеть примерно так: 
    
    ![Untitled](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/557ecedc-58cb-42da-8fee-88675f378145/Untitled.png)
    
    Чтобы перечислить несколько стран, нужно пользоваться такой конструкцией как
    WHERE Country IN ('A', 'B', 'C')

In [ ]:
SELECT Country,
    toStartOfMonth(InvoiceDate) AS InvoiceDate_Month,
    COUNT(DISTINCT(CustomerID)) AS Count_CustomerID
FROM default.retail
WHERE Country IN ('United Kingdom', 'Netherlands', 'Australia')
GROUP BY Country,InvoiceDate_Month

##### ОТВЕТ: 2

## 5

In [ ]:
Как вы могли заметить, график с предыдущего шага получается не очень информативным для 2 из 3 стран.

Давайте посмотрим на динамику изменения числа активных пользователей в месяц уже для всех стран, кроме Великобритании. 

В каких месяцах число активных пользователей во Франции было больше, чем в Германии?

In [ ]:
SELECT Country,
    toStartOfMonth(InvoiceDate) AS InvoiceDate_Month,
    COUNT(DISTINCT(CustomerID)) AS Count_CustomerID
FROM default.retail
WHERE Country  NOT IN ('United Kingdom')
GROUP BY Country,InvoiceDate_Month

##### ОТВЕТ :
Сентябрь 2011
Декабрь 2011
Май 2011


## 6

In [ ]:
Теперь проанализируем сами заказы. Посчитайте среднюю сумму заказа (AOV – average order value) в каждой из стран.

Обратите внимание, что пользователь может добавить несколько одинаковых товаров в один заказ, 

и для подсчета суммы каждого заказа можно создать столбец TotalPrice, в котором будет храниться цена, 

которую пользователь заплатил, например, за три упаковки чая.

InvoiceNo 	Description 	Quantity 	UnitPrice	TotalPrice
12345 	12345 	3	1.2	3.6
12345 	COOKIES 	1	2.5	2.5
Далее для каждого заказа нужно просуммировать полученную колонку, а затем сгруппировать данные по странам и 

посчитать среднюю сумму заказа в каждой из них.

Визуализируйте результат. Из представленных вариантов ответа выберите страны, входящие в топ-3 по величине средней суммы заказа.

NB! В таблице есть отрицательные значения Quantity, что соответствует отмененным заказам. Их в расчет AOV включать не нужно. 

In [ ]:
SELECT Country,
    AVG(UnitPrice*Quantity) AS AOV
    
FROM default.retail
WHERE Quantity > 0

GROUP BY Country

In [ ]:
SELECT Country,
    AVG(SUM_InvoiceNo) AS AOV

FROM
    (
    SELECT Country,
        InvoiceNo,
        SUM(UnitPrice * Quantity) AS SUM_InvoiceNo
    FROM default.retail
    WHERE Quantity > 0
    GROUP BY Country, InvoiceNo
    )
GROUP BY Country
LIMIT 100

##### ОТВЕТ: 
Australia
Singapore
Netherlands

## 7

In [ ]:
Как изменялась средняя сумма заказа в разных странах по месяцам?

Предположим, нас интересует динамика в следующих странах: 
United Kingdom, Germany, France, Spain, Netherlands, Belgium, Switzerland, Portugal, Australia, USA. 
Визуализируйте результат и выберите верные утверждения.

 

In [ ]:
SELECT Country,
    AVG(SUM_InvoiceNo) AS Total_Sum_day,
    InvoiceDate_Month

FROM
    (
    SELECT Country,
        InvoiceNo,
        toStartOfMonth(InvoiceDate) AS InvoiceDate_Month,
        SUM(UnitPrice * Quantity) AS SUM_InvoiceNo
    FROM default.retail
    WHERE Quantity > 0 AND Country IN ('United Kingdom', 'Germany', 'France', 'Spain', 'Netherlands', 'Belgium', 'Switzerland', 'Portugal','Australia', 'USA')
    GROUP BY Country, InvoiceNo,InvoiceDate_Month
    )
GROUP BY Country,InvoiceDate_Month

##### ОТВЕТ:
Аномальный рост среднего чека произошел в августе 2011 среди пользователей из Австралии

Самые большие значения среднего чека наблюдаются в Австралии и Нидерландах

Где-то в октябре средний чек США подскочил вверх с нуля

## 8

In [ ]:
Сколько товаров пользователи обычно добавляют в корзину? Посчитайте среднее количество товаров, добавленных в корзину, 
с разбивкой по странам. 

В качестве ответа укажите среднее число товаров в заказе среди пользователей в Канаде (Canada). Разделитель - точка.

In [ ]:
# Сделал в табикс
SELECT Country,
    AVG(SUM_Quantity) AS Avg_SUM_Quantity

FROM
    (
    SELECT Country,
        InvoiceNo,
        SUM(Quantity) AS SUM_Quantity
    FROM default.retail
    WHERE Quantity > 0
    GROUP BY Country, InvoiceNo
    )
GROUP BY Country
LIMIT 100

In [ ]:
Sweden	1002.3055555555555
Bahrain	130
Japan	1369.2631578947369
Finland	261.0731707317073
Brazil	356
EIRE	540.4807692307693
Belgium	237.1122448979592
Israel	809.2
Norway	537.1666666666666
Portugal	282.8421052631579
Canada	460.5
France	286.560411311054
Malta	194
Poland	193.89473684210526
Italy	213.47368421052633
Greece	311.4
Spain	310.56666666666666

##### Ответ: 460.5

## 9

In [ ]:
Возможно, результат на предыдущем шаге показался вам странным, особенно если соотнести средний размер корзины с числом уникальных пользователей в некоторых странах. 

Посмотрим на Нидерланды (Netherlands) более подробно. Сгруппируйте данные по пользователям и посмотрите, 
кто купил наибольшее число товаров. В ответе укажите идентификатор данного пользователя. 

In [ ]:
# 7 урок > 9 шаг > SQL

- **Как решить это задание?**
    
    В этом задании нужно:
    
    1. Посчитать **количество товаров**, которые добавили в корзину, сгруппировав данные по пользователям
    2. В ответе укажите идентификатор данного пользователя, который купил **наибольшее число товаров** в **Нидерландах** 
    (Netherlands)

In [ ]:
SELECT CustomerID,
        SUM(Quantity) AS SUM_Quantity
    FROM default.retail
    WHERE Quantity > 0 AND Country = 'Netherlands'
GROUP BY CustomerID

ORDER BY CustomerID DESC

In [ ]:
14646	197491
12802	314
12791	97
12790	166
12789	4
12787	261
12778	825
12775	877
12759	902

##### ОТВЕТ : 14646

## 10

In [ ]:
Один из пользователей добавил к себе в корзину почти 200 тысяч товаров. Не кажется ли это подозрительным?

Изучите, что именно он покупает, посмотрите на количество товаров в каждом заказе, итоговые суммы. Подумайте, кто или что это может быть 🤔

Когда подумаете, просто введите идентификатор данного пользователя еще раз в поле ниже.

In [ ]:
В этом задании самая интересная часть аналитик, а именно - исследование аномалии.

1) Вы можете забрать все имеющиеся записи в retail по CustomerID этого пользователя, отсортировать результат по количеству купленного и посмотреть на таблицу, поискать закономерности.

2) Можно попробовать собрать самые покупаемые этим пользователем товары и разбить их на категории. Поискать закономерности в датах покупок, например.

3) Может, все эти платежи были оптовой закупкой? А может это баг и все эти покупки от множества пользователей но записались в базу под одним идентификатором.

##### ОТВЕТ : 14646 , похоже на магазин какой-то

## 11

In [ ]:
Задание на построение дашборда
Следующая задача: подготовить дашборд в Redash (состоящий из более чем четырх графиков). Есть несколько основных вариантов, и в обоих случаях вы можете дополнить его любыми графиками, которые кажутся вам интересными и полезными:

Построить дашборд для отдела продаж. Вывести основные показатели: например, выручку по странам, дням, топ товаров, и другие графики, которые вы считаете необходимыми для отражения текущего положения дел.
Самостоятельно поставить себе задачу :)  
В поле ниже добавьте следующую информацию:

Ссылку на ваш дашборд в Redash (формат: https://redash.lab.karpov.courses/dashboards/....)
Краткое описание и основные выводы (если вы выбрали свою тему, то желательно описать более подробно)

In [ ]:

# https://redash.lab.karpov.courses/dashboards/2792-marketing_kamsarin

In [ ]:
# Запрос на ТОП 10 стран по выручке
SELECT Country,
    SUM(SUM_InvoiceNo) AS annual_revenue
    
FROM
    (
    SELECT 
        CustomerID,
        Country,
        InvoiceNo,
        toStartOfMonth(InvoiceDate) AS InvoiceDate_Month,
        SUM(UnitPrice * Quantity) AS SUM_InvoiceNo
    FROM default.retail
    
    GROUP BY CustomerID,Country, InvoiceNo,InvoiceDate_Month
    )
GROUP BY Country
ORDER BY annual_revenue DESC
LIMIT 10

In [ ]:
# Запрос на ТОП 10 покупателей
SELECT 
    CustomerID,
    Country,
    SUM(SUM_InvoiceNo) AS order_amount
    

FROM
    (
    SELECT 
        CustomerID,
        Country,
       
        SUM(UnitPrice * Quantity) AS SUM_InvoiceNo
    FROM default.retail
    
    GROUP BY CustomerID,Country
    )
GROUP BY CustomerID, Country
ORDER BY order_amount DESC
LIMIT 10

In [ ]:
# Запрос на популярные товары
SELECT 
        Description,
        StockCode,
        COUNT(Quantity) as Total_quantity
        
FROM default.retail
WHERE Description != 'POSTAGE' and Quantity > 0
    
GROUP BY Description, StockCode
ORDER BY Total_quantity DESC
LIMIT 10

In [ ]:
# Запрос выручки по месяцам
SELECT InvoiceDate_Month,
    SUM(SUM_InvoiceNo) AS annual_revenue
    
FROM
    (
    SELECT 
        CustomerID,
        Country,
        InvoiceNo,
        toStartOfMonth(InvoiceDate) AS InvoiceDate_Month,
        SUM(UnitPrice * Quantity) AS SUM_InvoiceNo
    FROM default.retail
    
    GROUP BY CustomerID,Country, InvoiceNo,InvoiceDate_Month
    )
GROUP BY InvoiceDate_Month

In [ ]:
# Запрос выручки по дням недели
SELECT InvoiceDate_Day,
    SUM(SUM_InvoiceNo) AS Sum_Day
    
FROM
    (
    SELECT 
        CustomerID,
        Country,
        InvoiceNo,
        toDayOfWeek(InvoiceDate) AS InvoiceDate_Day,
        SUM(UnitPrice * Quantity) AS SUM_InvoiceNo
    FROM default.retail
    
    GROUP BY CustomerID,Country, InvoiceNo,InvoiceDate_Day
    )
GROUP BY InvoiceDate_Day
ORDER BY InvoiceDate_Day

In [ ]:
# Запрос текущего времени
SELECT now();